#  Lottery Notebook

### imports :

In [34]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import numpy as np
import torch.nn.utils.prune as prune
import matplotlib.pyplot as plt
from pruning import Pruning_tool
import copy
import time
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print('using', device)

using cuda:0


## Model :

In [42]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc2 = nn.Linear(400, 200)
        self.fc3 = nn.Linear(200,100)
        self.fc41 = nn.Linear(100, 32)
        self.fc42 = nn.Linear(100, 32)
        self.fc5 = nn.Linear(32, 100)
        self.fc6 = nn.Linear(100,200)
        self.fc7 = nn.Linear(200,400)
        self.fc8 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc3(self.fc2(self.fc1(x))))
        return self.fc41(h1), self.fc42(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc7(self.fc6(self.fc5(z))))
        return torch.sigmoid(self.fc8(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [43]:
epochs = 10
batch_size = 64
log_interval = 100

mnist_trainset = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
                   batch_size=batch_size, shuffle=True)

mnist_testset = test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
                  batch_size=batch_size, shuffle=True)

model = VAE().to(device)
print(model.parameters)
optimizer = optim.Adam(model.parameters(), lr=7e-4)
rewind_state_dict = copy.deepcopy(model.state_dict()) 
step = 0

<bound method Module.parameters of VAE(
  (fc1): Linear(in_features=784, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=100, bias=True)
  (fc41): Linear(in_features=100, out_features=32, bias=True)
  (fc42): Linear(in_features=100, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=100, bias=True)
  (fc6): Linear(in_features=100, out_features=200, bias=True)
  (fc7): Linear(in_features=200, out_features=400, bias=True)
  (fc8): Linear(in_features=400, out_features=784, bias=True)
)>


In [44]:
def kl_anneal_function(anneal_function, step, k, x0):
    """ Beta update function
        
        Parameters
        ----------
        anneal_function : string
            What type of update (logisitc or linear)
        step : int
            Which step of the training
        k : float
            Coefficient of the logistic function
        x0 : float
            Delay of the logistic function or slope of the linear function
        Returns
        -------
        beta : float
            Weight of the KL divergence in the loss function 
        """
    if anneal_function == 'logistic':
        return float(1/(1+np.exp(-k*(step-x0))))
    elif anneal_function == 'linear':
        return min(1, step/x0)



# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar, beta):
    """ Compute the loss function between recon_x (output of the VAE) 
    and x (input of the VAE)
    """
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = 0.5 * torch.sum(logvar.exp() + mu.pow(2) - 1 - logvar)
    return BCE + beta*KLD

In [47]:
plt_loss = []
for name, param in model.named_parameters(): 
    param.data = rewind_state_dict[name].clone()
for epoch in range(15): # epochs + 1):

    model.train()
    train_loss = 0

    for batch_idx, (data, _) in enumerate(mnist_trainset):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)

        beta = kl_anneal_function('linear', step, 1, 10*len(mnist_trainset))
            
        loss = loss_function(recon_batch, data, mu, logvar, beta)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        step += 1
        #if batch_idx == 500:
        #    rewind_state_dict = copy.deepcopy(model.state_dict())
        #    print('rewind state saved')

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(mnist_trainset.dataset),
                100. * batch_idx / len(mnist_trainset),
                loss.item() / len(data)))
            plt_loss.append(loss.item() / len(data))  # For ploting loss

    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(mnist_trainset.dataset)))


Train Epoch: 0 [0/60000 (0%)]	Loss: 444.865509
Train Epoch: 0 [6400/60000 (11%)]	Loss: 200.020233
Train Epoch: 0 [12800/60000 (21%)]	Loss: 212.490952
Train Epoch: 0 [19200/60000 (32%)]	Loss: 188.451279
Train Epoch: 0 [25600/60000 (43%)]	Loss: 184.191925
Train Epoch: 0 [32000/60000 (53%)]	Loss: 160.402390
Train Epoch: 0 [38400/60000 (64%)]	Loss: 169.397400
Train Epoch: 0 [44800/60000 (75%)]	Loss: 155.762924
Train Epoch: 0 [51200/60000 (85%)]	Loss: 156.988495
Train Epoch: 0 [57600/60000 (96%)]	Loss: 155.283737
====> Epoch: 0 Average loss: 180.5791
Train Epoch: 1 [0/60000 (0%)]	Loss: 145.979568
Train Epoch: 1 [6400/60000 (11%)]	Loss: 141.154205
Train Epoch: 1 [12800/60000 (21%)]	Loss: 133.171875
Train Epoch: 1 [19200/60000 (32%)]	Loss: 129.308990
Train Epoch: 1 [25600/60000 (43%)]	Loss: 136.864258
Train Epoch: 1 [32000/60000 (53%)]	Loss: 133.485840
Train Epoch: 1 [38400/60000 (64%)]	Loss: 136.080261
Train Epoch: 1 [44800/60000 (75%)]	Loss: 123.717049
Train Epoch: 1 [51200/60000 (85%)]	Los

Train Epoch: 14 [57600/60000 (96%)]	Loss: 108.944412
====> Epoch: 14 Average loss: 110.5062


## Pruning


In [22]:
from pruning import Pruning
from pruning import PruningBack


#local pruning
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        pr = Pruning(module)
        prb = PruningBack(module)
        pr.set_mask(0.7)
        prb.set_mask(0.7)

        module.register_forward_pre_hook(pr)
        #module.register_backward_hook(prb)

for batch_idx, (data, _) in enumerate(mnist_trainset):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        break

prstat = Pruning_tool()
prstat.stats_pruning(model)

Model :
Linear(in_features=784, out_features=400, bias=True)
Sparsity in Layer 0: 50.00%
Linear(in_features=400, out_features=40, bias=True)
Sparsity in Layer 1: 50.01%
Linear(in_features=400, out_features=40, bias=True)
Sparsity in Layer 2: 50.01%
Linear(in_features=40, out_features=400, bias=True)
Sparsity in Layer 3: 50.01%
Linear(in_features=400, out_features=784, bias=True)
Sparsity in Layer 4: 50.00%
Global Sparsity : 50.00%


In [46]:
from pruning import Pruning
from pruning import PruningBack
plt_loss = []
prstat = Pruning_tool()

pruning_amount = np.arange(10)/10

for pa in pruning_amount:
    print(pa*100,'% of weights pruned')
    #local pruning
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            pr = Pruning(module)
            prb = PruningBack(module)
            pr.set_mask(pa)
            prb.set_mask(pa)

            module.register_forward_pre_hook(pr)
            #module.register_backward_hook(prb)

    for name, param in model.named_parameters(): 
        param.data = rewind_state_dict[name].clone()
    for epoch in range(0, 3):

        model.train()
        train_loss = 0
        for batch_idx, (data, _) in enumerate(mnist_trainset):
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)

            beta = kl_anneal_function('linear', step, 1, 10*len(mnist_trainset))
            #print(beta)
            loss = loss_function(recon_batch, data, mu, logvar, beta)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
            step += 1

            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(mnist_trainset.dataset),
                    100. * batch_idx / len(mnist_trainset),
                    loss.item() / len(data)))
                plt_loss.append(loss.item() / len(data))  # For ploting loss
        print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(mnist_trainset.dataset)))
    _, _, _ = model(data)
    prstat.stats_pruning(model)

prstat.stats_pruning(model)   

0.0 % of weights pruned
Train Epoch: 0 [0/60000 (0%)]	Loss: 131.466370
Train Epoch: 0 [6400/60000 (11%)]	Loss: 115.765579
Train Epoch: 0 [12800/60000 (21%)]	Loss: 119.548943
Train Epoch: 0 [19200/60000 (32%)]	Loss: 119.671738
Train Epoch: 0 [25600/60000 (43%)]	Loss: 123.205589
Train Epoch: 0 [32000/60000 (53%)]	Loss: 112.860954
Train Epoch: 0 [38400/60000 (64%)]	Loss: 113.997025
Train Epoch: 0 [44800/60000 (75%)]	Loss: 110.145615
Train Epoch: 0 [51200/60000 (85%)]	Loss: 108.668976
Train Epoch: 0 [57600/60000 (96%)]	Loss: 108.028481
====> Epoch: 0 Average loss: 113.9240
Train Epoch: 1 [0/60000 (0%)]	Loss: 99.202461
Train Epoch: 1 [6400/60000 (11%)]	Loss: 111.322105
Train Epoch: 1 [12800/60000 (21%)]	Loss: 102.749695
Train Epoch: 1 [19200/60000 (32%)]	Loss: 103.129349
Train Epoch: 1 [25600/60000 (43%)]	Loss: 96.888809
Train Epoch: 1 [32000/60000 (53%)]	Loss: 98.240273
Train Epoch: 1 [38400/60000 (64%)]	Loss: 96.409241
Train Epoch: 1 [44800/60000 (75%)]	Loss: 96.462883
Train Epoch: 1 [512

Train Epoch: 1 [44800/60000 (75%)]	Loss: 112.936470
Train Epoch: 1 [51200/60000 (85%)]	Loss: 113.707306
Train Epoch: 1 [57600/60000 (96%)]	Loss: 110.346634
====> Epoch: 1 Average loss: 113.5518
Train Epoch: 2 [0/60000 (0%)]	Loss: 106.689796
Train Epoch: 2 [6400/60000 (11%)]	Loss: 111.876030
Train Epoch: 2 [12800/60000 (21%)]	Loss: 115.199852
Train Epoch: 2 [19200/60000 (32%)]	Loss: 118.048347
Train Epoch: 2 [25600/60000 (43%)]	Loss: 110.380135
Train Epoch: 2 [32000/60000 (53%)]	Loss: 107.612885
Train Epoch: 2 [38400/60000 (64%)]	Loss: 111.636246
Train Epoch: 2 [44800/60000 (75%)]	Loss: 115.421379
Train Epoch: 2 [51200/60000 (85%)]	Loss: 109.992386
Train Epoch: 2 [57600/60000 (96%)]	Loss: 110.403625
====> Epoch: 2 Average loss: 110.4400
Model :
Linear(in_features=784, out_features=400, bias=True)
Sparsity in Layer 0: 30.00%
Linear(in_features=400, out_features=200, bias=True)
Sparsity in Layer 1: 30.00%
Linear(in_features=200, out_features=100, bias=True)
Sparsity in Layer 2: 30.00%
Lin

Train Epoch: 0 [6400/60000 (11%)]	Loss: 146.366898
Train Epoch: 0 [12800/60000 (21%)]	Loss: 131.893097
Train Epoch: 0 [19200/60000 (32%)]	Loss: 131.008423
Train Epoch: 0 [25600/60000 (43%)]	Loss: 122.806183
Train Epoch: 0 [32000/60000 (53%)]	Loss: 119.148315
Train Epoch: 0 [38400/60000 (64%)]	Loss: 114.202148
Train Epoch: 0 [44800/60000 (75%)]	Loss: 113.233658
Train Epoch: 0 [51200/60000 (85%)]	Loss: 118.709518
Train Epoch: 0 [57600/60000 (96%)]	Loss: 113.151505
====> Epoch: 0 Average loss: 127.7825
Train Epoch: 1 [0/60000 (0%)]	Loss: 110.391785
Train Epoch: 1 [6400/60000 (11%)]	Loss: 111.289551
Train Epoch: 1 [12800/60000 (21%)]	Loss: 114.103569
Train Epoch: 1 [19200/60000 (32%)]	Loss: 103.167053
Train Epoch: 1 [25600/60000 (43%)]	Loss: 113.470070
Train Epoch: 1 [32000/60000 (53%)]	Loss: 119.172699
Train Epoch: 1 [38400/60000 (64%)]	Loss: 101.760689
Train Epoch: 1 [44800/60000 (75%)]	Loss: 107.062057
Train Epoch: 1 [51200/60000 (85%)]	Loss: 107.633301
Train Epoch: 1 [57600/60000 (96%)